# RSS Infomation Research

In [5]:
# RSS에서 xml태그 별 정보 추출
!pip install feedparser 

# 인터넷 신문기사 분석(라이브러리 내의 "Article()"사용하기 위해)
!pip install newspaper3k 

# 한국어 형태소 분석기 (주어진 문장에서 명사만 추출할 목적으로 사용)
!pip install konlpy 

# 라이브러리 불러오기

In [6]:
import feedparser # xml 태그별 정보 추출
from newspaper import Article # 인터넷 신문 기사 분석
from konlpy.tag import Okt # 한국어 형태소 분석기
from collections import Counter # 갯수 세기(TF계산용)
from operator import eq # 동일한 단어인지 여부 확인 위해 사용
from bs4 import BeautifulSoup # 글에 존재할지 모르는 html 태그 삭제를 위해 사용

# 정보 수집할 rss 링크 설정 및 정보 수집하기

##### 전자신문 (https://www.etnews.com/rss/) 의 오늘의 뉴스 부분 

##### 함수 정의

In [7]:
# 제목, 링크 수집 함수
def crawl_rss(urls):
    array_rss = [] # 모든 기사를 넣을 리스트
    set_link = set() # 중복된 기사를 제거하기 위한 기사 제목들 집합
    
    for url in urls: # url 순회
        print(f"[Crawl Rss] {url}") # 현재 크롤링하는 rss 링크 출력
        parser_rss = feedparser.parse(url) # 현재 url을 파싱해서, 그 결과를 저장
        for p in parser_rss.entries:
            if p.link not in set_link: # 중복되지 않은 기사 일 경우 if 문 실행
                array_rss.append({'link':p.link})
                set_link.add(p.link)
    return array_rss

# 링크를 통해 제목, 본문 내용 추출
def crawl_article(url, language='ko'):
    print(f"[Crawling. . .] {url}") # 현재 크롤링하는 링크 출력
    news_paper = Article(url, language=language)
    news_paper.download()
    news_paper.parse()
    return news_paper.title, preprocessing(news_paper.text)

# 불용어 전처리
def preprocessing(text): 
    text_article = BeautifulSoup(text, 'html5lib').get_text() # html 태그 제거하기
    return text_article

In [8]:
# 정보 수집할 rss 링크 모음
urls = ['http://rss.etnews.com/Section901.xml',
        'http://rss.etnews.com/Section902.xml',
        'http://rss.etnews.com/Section903.xml',
        'http://rss.etnews.com/Section904.xml']

# 제목과 기사 링크 수집하기
list_articles = crawl_rss(urls)

# 수집한 (제목,기사링크)를 토대로 제목과 본문 내용 수집하기
for article in list_articles:
    title, text = crawl_article(article['link'])
    article['title'] = title
    article['text'] = text

[Crawl Rss] http://rss.etnews.com/Section901.xml
[Crawl Rss] http://rss.etnews.com/Section902.xml
[Crawl Rss] http://rss.etnews.com/Section903.xml
[Crawl Rss] http://rss.etnews.com/Section904.xml
[Crawling. . .] https://www.etnews.com/20231004000087
[Crawling. . .] https://www.etnews.com/20231004000053
[Crawling. . .] https://www.etnews.com/20231004000021
[Crawling. . .] https://www.etnews.com/20231003000006
[Crawling. . .] https://www.etnews.com/20230927000112
[Crawling. . .] https://www.etnews.com/20231004000004
[Crawling. . .] https://www.etnews.com/20231004000003
[Crawling. . .] https://www.etnews.com/20231004000002
[Crawling. . .] https://www.etnews.com/20231003000103
[Crawling. . .] https://www.etnews.com/20230926000252
[Crawling. . .] https://www.etnews.com/20231002000054
[Crawling. . .] https://www.etnews.com/20231002000053
[Crawling. . .] https://www.etnews.com/20230926000326
[Crawling. . .] https://www.etnews.com/20230926000306
[Crawling. . .] https://www.etnews.com/202309260

[Crawling. . .] https://www.etnews.com/20230930000031
[Crawling. . .] https://www.etnews.com/20230930000032
[Crawling. . .] https://www.etnews.com/20230927000060


# 수집 내용 확인하기

In [9]:
list_articles[0]

{'link': 'https://www.etnews.com/20231004000087',
 'title': "빗썸 메타버스 출격준비 완료…16조 시장 '패션·데이팅' 잡는다",
 'text': "출시를 앞둔 빗썸의 메타버스 플랫폼 '네모월드'가 패션 커머스 시장과 데이팅 플랫폼 시장을 정조준한다. 기존에 존재하던 웹 2.0 서비스를 화려한 3D 그래픽의 '메타 커머스' 플랫폼으로 그대로 옮겨온다는 구상이다.\n\n\n\n업계에 따르면 빗썸의 웹3.0 메타버스 플랫폼 자회사 빗썸메타(대표 조현식)는 패션 브랜드사와 협업한 디지털 쇼룸과 커머스 시스템을 네모월드 내에 구현, 이르면 올해 4분기 소프트론칭한다. 내년 공식 출시까지 20여개 브랜드를 메타버스로 가져오는 것이 목표다.\n\n\n\n단순히 메타버스 내 사이버 상품만 구입하는 것이 아니라, 현실세계에서 이용자가 입을 패션 상품도 함께 구입하는 '메타 커머스' 플랫폼을 구현한다. 대체불가토큰(NFT) 시스템을 적용해 한정판 구입 고객만 초대받을 수 있는 패션쇼나 라운지 개념도 검토 중이다.\n\n\n\n이를 활용해 메타버스 내에서 이용자들이 교류할 수 있는 소셜 스페이스 플랫폼으로 확장한다. 틴더와 같은 데이팅 앱, 사회관계망서비스(SNS)가 갖고 있는 기능을 아바타 시스템을 통해 가져오겠다는 계획이다. 이처럼 메타버스 수용(Metaverse-Adoption) 가능한 웹2.0 서비스는 커머스가 약 8.4조원, 소셜스페이스 시장이 약 7.5조원 수준으로 추산된다.\n\n\n\n빗썸메타 네모월드 지향점은 '실사와 같은 3D 메타버스'다. 게임개발에 사용되는 '언리얼5 엔진'을 기반으로 아바타, 공간, 상품을 실제 수준에 가깝게 그려냈다. 특히 주력 타깃 시장인 패션상품의 경우 현실 상품의 재질감까지 구현했다. 성인들도 메타버스 세계관에 재미와 몰입감을 느낄 수 있을뿐만 아니라, 구매 소비와 경제활동으로 이어질 수 있을 가능성을 염두했다.\n\n\n\n앞서 빗썸메타는 지난달 국내 최대 패션 박람회인 '프리뷰 인 

# 모든 본문 text에서 명사  (키워드, 빈도수) 추출

##### 키워드와 빈도수 추출하는 함수 정의

In [10]:
def get_keywords_and_frequency(text,nKeywords=10): #텍스트와  추출할 키워드 개수
    spliter = Okt() # Okt 객체 생성
    nouns = spliter.nouns(text) # 명사 추출 
    count = Counter(nouns) # 명사별 빈도수 추출
    list_keywords = []
    
    for n,c in count.most_common(nKeywords): # 최빈값 높은 명사 순으로, nKeywords까지 출력
        item = {'keyword':n, 'count':c}
        list_keywords.append(item)
    return list_keywords

##### 추출 내용 저장하기

In [11]:
for article in list_articles:
    keywords = get_keywords_and_frequency(article['text'])
    article['keywords'] = keywords

# 검색어를 입력받아서 그 검색어를 가지고 있는 기사를 출력하기

In [12]:
# 검색어와 동일한 키워드가 있을 때, 빈도수 반환
def search_articles(query, list_keywords): # 쿼리가 키워드 리스트에 있으면 출력 (없으면 0)
    nWords = 0 # 키워드 빈도수 초기화
    for kw in list_keywords:
        if eq(query, kw['keyword']):
            nWords = kw['count']
    return nWords

In [13]:
query = input("키워드를 입력하세요 : ")

for article in list_articles:
    nQuery = search_articles(query, article['keywords'])
    if nQuery != 0:
        print(f"[TF] {nQuery}, [Title] {article['title']}, [URL] {article['link']}")

키워드를 입력하세요 : 반도체
[TF] 17, [Title] [미래 반도체 유니콘 열전]〈4〉퀄리타스반도체, 대규모 R&D로 '글로벌 인터커넥트 IP 기업' 도전, [URL] https://www.etnews.com/20231003000006
[TF] 6, [Title] 산업부, '무역·기술 안보 종합전략' 세운다…연내 포럼 구성, [URL] https://www.etnews.com/20231004000002
[TF] 15, [Title] 경북도, 4일 반도체 특화단지 추진단 출범, [URL] https://www.etnews.com/20231004000062
[TF] 12, [Title] 美, 반도체 투자 3억달러 미만 기업도 지원금…산업부 “韓 기업 지원”, [URL] https://www.etnews.com/20230930000218


In [14]:
list_articles[0]

{'link': 'https://www.etnews.com/20231004000087',
 'title': "빗썸 메타버스 출격준비 완료…16조 시장 '패션·데이팅' 잡는다",
 'text': "출시를 앞둔 빗썸의 메타버스 플랫폼 '네모월드'가 패션 커머스 시장과 데이팅 플랫폼 시장을 정조준한다. 기존에 존재하던 웹 2.0 서비스를 화려한 3D 그래픽의 '메타 커머스' 플랫폼으로 그대로 옮겨온다는 구상이다.\n\n\n\n업계에 따르면 빗썸의 웹3.0 메타버스 플랫폼 자회사 빗썸메타(대표 조현식)는 패션 브랜드사와 협업한 디지털 쇼룸과 커머스 시스템을 네모월드 내에 구현, 이르면 올해 4분기 소프트론칭한다. 내년 공식 출시까지 20여개 브랜드를 메타버스로 가져오는 것이 목표다.\n\n\n\n단순히 메타버스 내 사이버 상품만 구입하는 것이 아니라, 현실세계에서 이용자가 입을 패션 상품도 함께 구입하는 '메타 커머스' 플랫폼을 구현한다. 대체불가토큰(NFT) 시스템을 적용해 한정판 구입 고객만 초대받을 수 있는 패션쇼나 라운지 개념도 검토 중이다.\n\n\n\n이를 활용해 메타버스 내에서 이용자들이 교류할 수 있는 소셜 스페이스 플랫폼으로 확장한다. 틴더와 같은 데이팅 앱, 사회관계망서비스(SNS)가 갖고 있는 기능을 아바타 시스템을 통해 가져오겠다는 계획이다. 이처럼 메타버스 수용(Metaverse-Adoption) 가능한 웹2.0 서비스는 커머스가 약 8.4조원, 소셜스페이스 시장이 약 7.5조원 수준으로 추산된다.\n\n\n\n빗썸메타 네모월드 지향점은 '실사와 같은 3D 메타버스'다. 게임개발에 사용되는 '언리얼5 엔진'을 기반으로 아바타, 공간, 상품을 실제 수준에 가깝게 그려냈다. 특히 주력 타깃 시장인 패션상품의 경우 현실 상품의 재질감까지 구현했다. 성인들도 메타버스 세계관에 재미와 몰입감을 느낄 수 있을뿐만 아니라, 구매 소비와 경제활동으로 이어질 수 있을 가능성을 염두했다.\n\n\n\n앞서 빗썸메타는 지난달 국내 최대 패션 박람회인 '프리뷰 인 

# 검색어를 입력받아서 TF-IDF 구현 및 출력

In [15]:
from math import log # IDF 계산을 위해

In [16]:
query = input("키워드를 입력하세요 : ")
print()

sorted_list_articles = []
docs_count=0

for article in list_articles:
    nQuery = search_articles(query, article['keywords'])
    if nQuery != 0:
        docs_count+=1
        
    sorted_list_articles.append([article,nQuery])
#         print(f"[TF] {nQuery}, [Title] {article['title']}, [URL] {article['link']}")

# 정렬하여 출력
for i in sorted(sorted_list_articles, key=lambda x:x[1], reverse=True):
    
    tf = i[1]
    idf = log(len(sorted_list_articles)/(docs_count+1))
    tfidf = tf * idf # tfidf 구현
    
    print(f"[TF] {i[1]}, [IDF] {idf}, [TF-IDF] {tfidf}, [Title] {i[0]['title']}, [URL] {i[0]['link']}")
    

키워드를 입력하세요 : 반도체

[TF] 17, [IDF] 2.7212954278522306, [TF-IDF] 46.26202227348792, [Title] [미래 반도체 유니콘 열전]〈4〉퀄리타스반도체, 대규모 R&D로 '글로벌 인터커넥트 IP 기업' 도전, [URL] https://www.etnews.com/20231003000006
[TF] 15, [IDF] 2.7212954278522306, [TF-IDF] 40.819431417783456, [Title] 경북도, 4일 반도체 특화단지 추진단 출범, [URL] https://www.etnews.com/20231004000062
[TF] 12, [IDF] 2.7212954278522306, [TF-IDF] 32.65554513422677, [Title] 美, 반도체 투자 3억달러 미만 기업도 지원금…산업부 “韓 기업 지원”, [URL] https://www.etnews.com/20230930000218
[TF] 6, [IDF] 2.7212954278522306, [TF-IDF] 16.327772567113385, [Title] 산업부, '무역·기술 안보 종합전략' 세운다…연내 포럼 구성, [URL] https://www.etnews.com/20231004000002
[TF] 0, [IDF] 2.7212954278522306, [TF-IDF] 0.0, [Title] 빗썸 메타버스 출격준비 완료…16조 시장 '패션·데이팅' 잡는다, [URL] https://www.etnews.com/20231004000087
[TF] 0, [IDF] 2.7212954278522306, [TF-IDF] 0.0, [Title] 아이파트너즈, 혁신 스타트업 유럽진출 지원 시동, [URL] https://www.etnews.com/20231004000053
[TF] 0, [IDF] 2.7212954278522306, [TF-IDF] 0.0, [Title] 롯데칠성음료, 필리핀펩시 경영권 취득...글로벌 음료시장 공략, [URL]